<a href="https://colab.research.google.com/github/gguillaux/ml_python/blob/master/fiap_modelos_de_ia_e_ml_exercicio_1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Exercício #1.2

Utilizando o arquivo acidentes-apriori.csv.csv que é um recorte da base
de dados abertas dos acidentes de todas as rodovias federais, disponível
de forma completa em: https://portal.prf.gov.br/dados-abertos-acidentes
pela Polícia Rodoviária Federal.
Procure analisar quais são as regras mais relevantes para análise de
acidentes, tentem–se colocar no lugar no lugar de cientista de dados da
polícia rodoviária, quais são as regras relevantes? Existem itemsets
pouco relevantes que poderiam ser descartados na análise?
Realize alguns filtros nos itemsets para facilitar a análise das regras
geradas.

In [0]:
import os
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
# carregar csv
pf = pd.read_csv("./drive/My Drive/fiap/datasets/acidentes-apriori.csv",
                   sep=";", encoding='latin1')
pf.head()

,dia_semana,uf,causa_acidente,tipo_acidente,classificacao_acidente,fase_dia,condicao_metereologica,tracado_via
0,terca-feira,SP,Falta de Atencao Conducao,Colisao com objeto estatico,Com Vitimas Feridas,Plena Noite,Ceu Claro,Curva
1,terca-feira,PR,Falta de Atencao Conducao,Colisao traseira,Com Vitimas Feridas,Plena Noite,Nublado,Reta
2,terca-feira,SC,Animais na Pista,Colisao com objeto estatico,Com Vitimas Feridas,Plena Noite,Ceu Claro,Reta
3,terca-feira,CE,Ingestao de Substancias Psicoativas,Colisao com objeto estatico,Com Vitimas Feridas,Plena Noite,Nublado,Viaduto
4,terca-feira,MG,Falta de Atencao Conducao,Colisao transversal,Com Vitimas Feridas,Plena Noite,Nublado,Intersecao de vias


In [3]:
# analise da estrutura do dataset
cols = pf.columns
total_categories = 0
for col in cols:
    print('Valores da Coluna = {}'.format(col.upper()))
    k = pf[col].value_counts()
    total_categories += len(k)
    print(k)
    print('\n')
print('TOTAL OF CATEGORICAL VALUES {}'.format(total_categories))

Valores da Coluna = DIA_SEMANA
sabado           3491
domingo          3384
sexta-feira      3123
segunda-feira    2815
terca-feira      2771
quinta-feira     2703
quarta-feira     2595
Name: dia_semana, dtype: int64


Valores da Coluna = UF
MG    2717
SC    2636
PR    2411
RJ    1448
RS    1389
SP    1377
BA    1047
GO    1041
ES     871
PE     858
MT     659
CE     538
PB     516
RO     443
MS     431
PI     415
RN     413
MA     342
DF     300
PA     248
SE     213
AL     211
TO     161
RR      70
AC      60
AP      41
AM      26
Name: uf, dtype: int64


Valores da Coluna = CAUSA_ACIDENTE
Falta de Atencao Conducao                                                           7583
Desobediencia nas normas de transito pelo condutor                                  2391
Velocidade Incompativel                                                             1984
Ingestao de Alcool                                                                  1597
Nao guardar distancia de seguranca            

In [4]:
# exibir estrutura do df antes do one-hot-encoding
pf.shape

(20882, 8)

## Conclusao sobre a estrutura do dataset
Ele é composto inteiramente de dados categoricos.
Será preciso fazer um trabalho de pré-processamento para deixar
os dados em formato de one-hot-encoding. No total o dataframe resultante tera 98 colunas.

In [5]:
df_coded = pd.get_dummies(pf, prefix_sep='|')
display(df_coded)
print('\nResulting shape = {}'.format(df_coded.shape))

,dia_semana|domingo,dia_semana|quarta-feira,dia_semana|quinta-feira,dia_semana|sabado,dia_semana|segunda-feira,dia_semana|sexta-feira,dia_semana|terca-feira,uf|AC,uf|AL,uf|AM,uf|AP,uf|BA,uf|CE,uf|DF,uf|ES,uf|GO,uf|MA,uf|MG,uf|MS,uf|MT,uf|PA,uf|PB,uf|PE,uf|PI,uf|PR,uf|RJ,uf|RN,uf|RO,uf|RR,uf|RS,uf|SC,uf|SE,uf|SP,uf|TO,causa_acidente|Agressao Externa,causa_acidente|Animais na Pista,causa_acidente|Avarias e/ou desgaste excessivo no pneu,causa_acidente|Carga excessiva e/ou mal acondicionada,causa_acidente|Condutor Dormindo,causa_acidente|Defeito Mecanico no Veiculo,...,tipo_acidente|Atropelamento de Pedestre,tipo_acidente|Capotamento,tipo_acidente|Colisao com objeto em movimento,tipo_acidente|Colisao com objeto estatico,tipo_acidente|Colisao frontal,tipo_acidente|Colisao lateral,tipo_acidente|Colisao transversal,tipo_acidente|Colisao traseira,tipo_acidente|Danos eventuais,tipo_acidente|Derramamento de carga,tipo_acidente|Engavetamento,tipo_acidente|Incendio,tipo_acidente|Queda de ocupante de veiculo,tipo_acidente|Saida de leito carrocavel,tipo_acidente|Tombamento,classificacao_acidente|Com Vitimas Fatais,classificacao_acidente|Com Vitimas Feridas,classificacao_acidente|Sem Vitimas,fase_dia|Amanhecer,fase_dia|Anoitecer,fase_dia|Plena Noite,fase_dia|Pleno dia,condicao_metereologica|Ceu Claro,condicao_metereologica|Chuva,condicao_metereologica|Garoa/Chuvisco,condicao_metereologica|Ignorado,condicao_metereologica|Nevoeiro/Neblina,condicao_metereologica|Nublado,condicao_metereologica|Sol,condicao_metereologica|Vento,tracado_via|Curva,tracado_via|Desvio Temporario,tracado_via|Intersecao de vias,tracado_via|Nao Informado,tracado_via|Ponte,tracado_via|Reta,tracado_via|Retorno Regulamentado,tracado_via|Rotatoria,tracado_via|Tunel,tracado_via|Viaduto
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20877,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0
20878,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
20879,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
20880,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0



Resulting shape = (20882, 98)


In [0]:
frequencia = apriori(df_coded, 
                    min_support=0.05, 
                    use_colnames=True)

In [7]:
frequencia.sort_values(by='support', ascending=False)

,support,itemsets
28,0.760895,(classificacao_acidente|Com Vitimas Feridas)
39,0.610430,(tracado_via|Reta)
32,0.573221,(fase_dia|Pleno dia)
33,0.539556,(condicao_metereologica|Ceu Claro)
133,0.464371,"(tracado_via|Reta, classificacao_acidente|Com ..."
...,...,...
120,0.050714,"(fase_dia|Plena Noite, tipo_acidente|Saida de ..."
218,0.050330,"(classificacao_acidente|Com Vitimas Feridas, c..."
7,0.050139,(uf|BA)
170,0.050091,"(tracado_via|Reta, fase_dia|Pleno dia, dia_sem..."


In [0]:
def get_associations(df, min_confidence):
    associacoes = association_rules(df,
                                metric='confidence',
                                min_threshold=min_confidence)
    return associacoes.iloc[:, [0,1,4,5]].sort_values(by='confidence', 
                                                      ascending=False)

In [9]:
for i in np.linspace(0.5, 0.9, 5):
    _ = get_associations(frequencia, i)
    print('Regras geradas com {} % de min_confidence = {}'.format(i * 100, _.shape[0]))

Regras geradas com 50.0 % de min_confidence = 315
Regras geradas com 60.0 % de min_confidence = 193
Regras geradas com 70.0 % de min_confidence = 114
Regras geradas com 80.0 % de min_confidence = 43
Regras geradas com 90.0 % de min_confidence = 2


Dados os resultados acima de regras geradas, convem performar a analise apenas com regras de associação com confiança superior ou igual a 80 % visto que o universo de regras gerado é menor e mais passível de análise e atuação incisiva pelos membros da PRF.

Os demais resultados e regras, a despeito da importância estatística relativa, mereceriam maior crédito caso não houvesse trade-off de recursos.

In [0]:
assoc_80 = get_associations(frequencia, 0.8)

In [11]:
assoc_80.head(43).sort_values(by='support', ascending=False)

,antecedents,consequents,support,confidence
2,(causa_acidente|Falta de Atencao Conducao),(classificacao_acidente|Com Vitimas Feridas),0.295709,0.814322
25,"(tracado_via|Reta, causa_acidente|Falta de Ate...",(classificacao_acidente|Com Vitimas Feridas),0.193181,0.817427
22,"(fase_dia|Pleno dia, causa_acidente|Falta de A...",(classificacao_acidente|Com Vitimas Feridas),0.186189,0.827762
23,"(causa_acidente|Falta de Atencao Conducao, con...",(classificacao_acidente|Com Vitimas Feridas),0.168183,0.816364
39,"(tracado_via|Reta, fase_dia|Pleno dia, causa_a...",(classificacao_acidente|Com Vitimas Feridas),0.119960,0.830846
40,"(tracado_via|Reta, causa_acidente|Falta de Ate...",(classificacao_acidente|Com Vitimas Feridas),0.114740,0.819425
4,(tipo_acidente|Colisao transversal),(classificacao_acidente|Com Vitimas Feridas),0.109089,0.854784
38,"(fase_dia|Pleno dia, causa_acidente|Falta de A...",(classificacao_acidente|Com Vitimas Feridas),0.106934,0.828264
0,(uf|SC),(classificacao_acidente|Com Vitimas Feridas),0.102145,0.809181
1,(causa_acidente|Desobediencia nas normas de tr...,(classificacao_acidente|Com Vitimas Feridas),0.093095,0.813049


In [0]:
# remover consquente dia com sol, por nao ser tao relevante como consequente
dia_filter = assoc_80.consequents.apply(lambda x: 
                                        x.issuperset({'fase_dia|Pleno dia'})) 
assoc_final = assoc_80[~dia_filter].sort_values(by='support', ascending=False)

In [13]:
pd.set_option('max_colwidth', 200)
assoc_final.head(10)

,antecedents,consequents,support,confidence
2,(causa_acidente|Falta de Atencao Conducao),(classificacao_acidente|Com Vitimas Feridas),0.295709,0.814322
25,"(tracado_via|Reta, causa_acidente|Falta de Atencao Conducao)",(classificacao_acidente|Com Vitimas Feridas),0.193181,0.817427
22,"(fase_dia|Pleno dia, causa_acidente|Falta de Atencao Conducao)",(classificacao_acidente|Com Vitimas Feridas),0.186189,0.827762
23,"(causa_acidente|Falta de Atencao Conducao, condicao_metereologica|Ceu Claro)",(classificacao_acidente|Com Vitimas Feridas),0.168183,0.816364
39,"(tracado_via|Reta, fase_dia|Pleno dia, causa_acidente|Falta de Atencao Conducao)",(classificacao_acidente|Com Vitimas Feridas),0.119960,0.830846
40,"(tracado_via|Reta, causa_acidente|Falta de Atencao Conducao, condicao_metereologica|Ceu Claro)",(classificacao_acidente|Com Vitimas Feridas),0.114740,0.819425
4,(tipo_acidente|Colisao transversal),(classificacao_acidente|Com Vitimas Feridas),0.109089,0.854784
38,"(fase_dia|Pleno dia, causa_acidente|Falta de Atencao Conducao, condicao_metereologica|Ceu Claro)",(classificacao_acidente|Com Vitimas Feridas),0.106934,0.828264
0,(uf|SC),(classificacao_acidente|Com Vitimas Feridas),0.102145,0.809181
1,(causa_acidente|Desobediencia nas normas de transito pelo condutor),(classificacao_acidente|Com Vitimas Feridas),0.093095,0.813049


## Conclusão
Acima exibimos as 10 principais regras de associação que resultaram em acidentes com vítimas feridas. Os dados possuem um suporte máximo de 29%, ou seja, a primeira regra de associação esteve presente em quase 1/3 de todos os acidentes registrados, e confiança mínima de 80%, ou seja, em todos os registros de cada antecedente exibido em pelo menos 80% dos registros o consequente enunciado esteve presente.

Isolando o foco nas 5 primeiras regras, vemos que um fator largamente presente entre os acidentes é a falta de atenção. Ela também ocorre em 8 dos 10 primeiros antecedentes.

Uma recomendação para alguma campanha de redução de acidentes seria reforçar a mensagem de aumentar o nível de atenção no momento em que se dirige. Valeria também ressaltar que essa mensagem é válida não apenas quando se ririge à noite mas também em pleno dia, visto que a luminosidade também aparece como uma antecedente relevante. 

Por fim, vale notar que cerca de 10% dos acidentes registrados tem como antecedente o estado de Santa Catarina (SC). A análise acima poderia recomendar uma concentração maior de investimento na campanha nesse estado. 